# Obtaining Your Data

## Introduction
In this lesson, you'll synthesize many of your data loading skills you learned to date in order to merge multiple datasets from various sources.

## Objectives
You will be able to:
* Understand the ETL process and the steps it consists of
* Understand the challenges of working with data from multiple sources 

## Loading SQL DB to DataFrames
<img src="Database-Schema.png">

In [2]:
import sqlite3
import pandas as pd

# Create a connection
con = sqlite3.connect('data.sqlite')
# Create a cursor
cur = con.cursor()
# Select some data
cur.execute("""SELECT * FROM orders JOIN orderdetails USING(orderNumber);""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(df.shape)
df.head()

(2996, 11)


,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,,363,S18_1749,30,136.00,3
1,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,,363,S18_2248,50,55.09,2
2,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,,363,S18_4409,22,75.46,4
3,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,,363,S24_3969,49,35.29,1
4,10101,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128,S18_2325,25,108.06,4


In [6]:
# Create a connection
con = sqlite3.connect('data.sqlite')
# Create a cursor
cur = con.cursor()
# Select some data
cur.execute("""SELECT * FROM products;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(df.shape)
df.head()

(110, 9)


,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70
1,S10_1949,1952 Alpine Renault 1300,Classic Cars,1:10,Classic Metal Creations,Turnable front wheels; steering function; deta...,7305,98.58,214.30
2,S10_2016,1996 Moto Guzzi 1100i,Motorcycles,1:10,Highway 66 Mini Classics,"Official Moto Guzzi logos and insignias, saddl...",6625,68.99,118.94
3,S10_4698,2003 Harley-Davidson Eagle Drag Bike,Motorcycles,1:10,Red Start Diecast,"Model features, official Harley Davidson logos...",5582,91.02,193.66
4,S10_4757,1972 Alfa Romeo GTA,Classic Cars,1:10,Motor City Art Classics,Features include: Turnable front wheels; steer...,3252,85.68,136.00


## Merging Data

Recall that you can also join data from multiple tables in SQL.

In [7]:
# Create a connection
con = sqlite3.connect('data.sqlite')
# Create a cursor
cur = con.cursor()
# Select some data
cur.execute("""SELECT * FROM products
                        JOIN orderdetails
                        USING (productCode);""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(df.shape)
df.head()

(2996, 13)


,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP,orderNumber,quantityOrdered,priceEach,orderLineNumber
0,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70,10107,30,81.35,2
1,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70,10121,34,86.13,5
2,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70,10134,41,90.92,2
3,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70,10145,45,76.56,6
4,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70,10159,49,81.35,14


You can also merge data from a separate csv file. For example, say you take a separate data source regarding daily sales data for the various branches. You might first generate a view from our database:

In [8]:
# Create a connection
con = sqlite3.connect('data.sqlite')
# Create a cursor
cur = con.cursor()
# Select some data
cur.execute("""SELECT * FROM customers
                        JOIN orders
                        USING(customerNumber);""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(df.shape)
df.head()

(326, 19)


,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit,orderNumber,orderDate,requiredDate,shippedDate,status,comments
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",,Nantes,,44000,France,1370,21000.00,10123,2003-05-20,2003-05-29,2003-05-22,Shipped,
1,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",,Nantes,,44000,France,1370,21000.00,10298,2004-09-27,2004-10-05,2004-10-01,Shipped,
2,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",,Nantes,,44000,France,1370,21000.00,10345,2004-11-25,2004-12-01,2004-11-26,Shipped,
3,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,,Las Vegas,NV,83030,USA,1166,71800.00,10124,2003-05-21,2003-05-29,2003-05-25,Shipped,Customer very concerned about the exact color ...
4,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,,Las Vegas,NV,83030,USA,1166,71800.00,10278,2004-08-06,2004-08-16,2004-08-09,Shipped,


And then load the seperate datefile:

In [16]:
daily_sums = pd.read_csv('Daily_Sales_Summaries.csv')
daily_sums.tail()

,orderDate,min,max,sum,mean,std
260,2005-05-16,662.90,5000.50,23627.44,2625.271111,1408.572001
261,2005-05-17,481.50,7863.80,52420.07,3744.290714,2064.005664
262,2005-05-29,529.35,6329.95,49890.61,3326.040667,1827.498485
263,2005-05-30,891.50,4663.44,14447.17,2063.881429,1236.177149
264,2005-05-31,553.52,10072.00,70933.74,3733.354737,2335.560075


In [15]:
len(daily_sums)

265

In [11]:
len(df)

326

In [12]:
merged = pd.merge(df, daily_sums)

## Checking Merged Data

It's always good practice to check assumptions and preview transformed data views throughout your process. Let's take a look:

In [13]:
len(merged)

326

In [14]:
merged.tail()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,...,orderDate,requiredDate,shippedDate,status,comments,min,max,sum,mean,std
321,489,"Double Decker Gift Stores, Ltd",Smith,Thomas,(171) 555-7555,120 Hanover Sq.,,London,,WA1 1DP,...,2004-01-22,2004-01-28,2004-01-27,Shipped,Difficult to negotiate with customer. We need ...,1461.00,3217.46,7310.42,2436.806667,894.344201
322,495,Diecast Collectables,Franco,Valarie,6175552555,6251 Ingle Ln.,,Boston,MA,51003,...,2003-12-09,2003-12-17,2003-12-11,Shipped,Check on availability.,1291.92,6195.64,59265.14,3704.071250,1564.227089
323,495,Diecast Collectables,Franco,Valarie,6175552555,6251 Ingle Ln.,,Boston,MA,51003,...,2004-04-26,2004-05-03,2004-04-28,Shipped,,1018.71,5257.89,6276.60,3138.300000,2997.552925
324,496,Kelly's Gift Shop,Snowden,Tony,+64 9 5555500,Arenales 1938 3'A',,Auckland,,,...,2003-07-07,2003-07-16,2003-07-13,Shipped,,730.18,4936.80,32077.44,2467.495385,1382.146181
325,496,Kelly's Gift Shop,Snowden,Tony,+64 9 5555500,Arenales 1938 3'A',,Auckland,,,...,2004-12-16,2004-12-22,2004-12-18,Shipped,,1675.55,6307.50,52166.00,2898.111111,1050.057830


Pandas' `merge()` function conveniently uses common column names between the DataFrames as keys. You can always specifically specify what columns to join on by using the `on` keyword as in `pd.merge(df1, df2, on=[col1, col2])`. Unfortunately, columns that are not identically named beforehand will not work with this convenience method. Additionally, it is imperative to check the formatting of the join keys between the tables. A number formatted as a string can often ruin joins, and separate formatting conventions such as 'U.S.' versus 'USA' are also important preprocessing considerations before merging data files from various sources. In this case, everything worked smoothly, but it's good to keep in mind what problems may occur.

## Saving Transformed Data to File
Finally, we can save our transformed dataset.

In [17]:
merged.to_csv('Merged_Dataset.csv', index=False)

## Summary
Well done! In this lesson you reviewed merges, as well as potential pitfalls in merging datasets from different sources. In the next lab, you'll get some practice doing this as an initial step to a regression task.